In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import warnings
import contextlib
from datetime import datetime
import torch.multiprocessing as mp
import sys
mp.set_start_method('spawn')

In [2]:
from utils.datasets import *
from utils.eval import *
from utils.model import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_names = [
    "resnet50",
    "resnet18", "alexnet", "vgg16",
    "densenet", "inception", "googlenet", 
    "resnext50_32x4d", "wide_resnet50_2"
]

cuda


In [4]:
# freezed model
def get_original_model(net):
    if net == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        # models.resnet18(pretrained=True)
    elif net == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        # model = models.resnet18(pretrained=True)
    elif net == "alexnet":
        # model = models.alexnet(pretrained=True)
        model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
    elif net == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        # model = models.vgg16(pretrained=True)
    elif net == "densenet":
        model = models.densenet161(weights=models.DenseNet161_Weights.DEFAULT)
        # model = models.densenet161(pretrained=True)
    elif net == "inception":
        model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
        # model = models.inception_v3(pretrained=True)
    elif net == "googlenet":
        model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
        # model = models.googlenet(pretrained=True)
    elif net == "resnext50_32x4d":
        model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.DEFAULT)
        # model = models.resnext50_32x4d(pretrained=True)
    elif net == "wide_resnet50_2":
        model = models.wide_resnet50_2(weights=models.Wide_ResNet50_2_Weights.DEFAULT)
        # model = models.wide_resnet50_2(pretrained=True)
    else:
        raise ValueError("Invalid model name")
    for param in model.parameters():
        param.requires_grad = False
    # Remove the last fully connected layer
    if "inception" in net:
        model.aux_logits=False
        num_ftrs = model.fc.in_features
        model.fc = nn.Identity()
        model.fc = nn.Linear(2048, num_classes)
        optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
    if "alexnet" in net or "vgg" in net or  "mobilenet_v3_large" in net:
        last_fc_layer = model.classifier[-1]
        num_ftrs = last_fc_layer.in_features
        model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    elif "densenet" in net:
        last_fc_layer = model.classifier
        num_ftrs = last_fc_layer.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    else:
        num_ftrs = model.fc.in_features
        model.fc = nn.Identity()
        model.fc = nn.Linear(num_ftrs, num_classes)
        optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
                # Add padding to the convolutional layers if necessary
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    return model,criterion, optimizer

In [32]:
from pytorch_grad_cam import GradCAM, EigenCAM, AblationCAM, GradCAMPlusPlus, HiResCAM, XGradCAM, ScoreCAM, EigenGradCAM, FullGrad, LayerCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import numpy as np
import matplotlib.pyplot as plt
label_list = ["BMP4", "CHIR", "DS", "DS+CHIR",  "WT"]
def get_grad_cam(my_model, model_name, input_tensor):
    my_model.eval()
    if model_name in ["resnet50", "resnet18", "densenet", "resnext50_32x4d", "wide_resnet50_2"]:
        # For models with `layer4` as the last layer
        target_layer = my_model.layer4[-1].conv3
    elif model_name == "alexnet":
        # For AlexNet
        target_layer = my_model.features[-1]
    elif model_name == "vgg16":
        # For VGG16
        target_layer = my_model.features[-2]
    elif model_name == "inception":
        # For Inception
        target_layer = my_model.Mixed_7c
    elif model_name == "googlenet":
        # For GoogleNet
        target_layer = my_model.inception5b.branch4[1].conv
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    print(target_layer)
    # Create an instance of GradCAM
    cam = AblationCAM(model=my_model, target_layers=[target_layer], use_cuda=True)
    # Generate the CAM mask
    try:
        grayscale_cam = cam(input_tensor=input_tensor, targets=None, eigen_smooth=True)
    except Exception as e:
        print("Error in Grad-CAM computation:")
        raise e
        
    grayscale_cam = grayscale_cam[0, :]
    # cam_image = show_cam_on_image(input_tensor.cpu().numpy()[0].transpose(1, 2, 0), grayscale_cam, use_rgb=True)
    cam_image = show_cam_on_image(proprocess_image(input_tensor).cpu().numpy()[0].transpose(1, 2, 0), grayscale_cam, use_rgb=True)

    return cam_image

def display_heatmaps_for_timestamp(timestamp, model, model_name, dataloaders):
    model.eval()  # Ensure the model is in evaluation mode

    for inputs, labels in dataloaders['test']:
        for i in range(inputs.size(0)):
            # Get a single image from the batch
            input_tensor = inputs[i:i+1].to(device)
            label_tensor = labels[i:i+1].to(device)
            heatmap = get_grad_cam(model, model_name, input_tensor)
            original_image = proprocess_image(input_tensor).cpu().numpy()[0].transpose(1, 2, 0)
            outputs = model(input_tensor)
            _, predicted = torch.max(outputs.data, 1)
            predicted_label_class = label_list[predicted.cpu().numpy()[0]]
            actual_label_class = label_list[label_tensor.cpu().numpy()[0]]

            

            fig, axs = plt.subplots(1, 2, figsize=(20, 10))
            axs[0].imshow(original_image)
            axs[0].set_title(f"Original Image - Timestamp {timestamp} label: {actual_label_class}")
            
            axs[1].imshow(heatmap)
            axs[1].set_title(f"Heatmap - Timestamp {timestamp} label: {predicted_label_class}")
            
            plt.tight_layout()  # Adjust layout to prevent overlap
            save_dir = f"output/tl/{model_name}/heatmaps"
            os.makedirs(f"output/tl/{model_name}/heatmaps", exist_ok=True)
            plt.savefig(f"{save_dir}/timestamp_{timestamp}_{i}_pred_{predicted_label_class}_actual_{actual_label_class}")
            plt.close()

In [25]:
def load_model_for_timestamp(model_name, timestamp):
    num_epochs = 10
    # save_dir = os.path.join("saved_models/tl+rotation", "2024-02-20")
    save_dir = os.path.join("saved_models/tl", "2024-02-20")
    print("timestamp: ", timestamp)
    model, _, _ = get_original_model(model_name)
    model_filename = os.path.join(save_dir, f"model_{model_name}_time_{timestamp}_epochs{num_epochs}.pt")
    model.load_state_dict(torch.load(model_filename))
    return model

In [26]:
def read_metrics(file_path):
    avg_acc_map = {}
    avg_pre_map = {}
    avg_rec_map = {}
    avg_f1_map = {}
    acc_map = {}
    pre_map = {}
    rec_map = {}
    f1_map = {}
    tps_map = {}
    tns_map = {}
    fps_map = {}
    fns_map = {}

    current_model = None

    with open(file_path, "r") as file:
        current_model = None
        for _ in range(len(model_names)):
            next(file)
        for line in file:
            line = line.strip()
            if line:
                if not line.startswith("Accuracy:"):
                    current_model = line
                else:
                    acc_values = line.split(": ")[1].strip()[1:-1].split(", ")
                    acc_map[current_model] = [float(value) for value in acc_values]
                    pre_values = file.readline().split(": ")[1].strip()[1:-1].split(", ")
                    pre_map[current_model] = [float(value) for value in pre_values]
                    rec_values = file.readline().strip().split(": ")[1].strip()[1:-1].split(", ")
                    rec_map[current_model] = [float(value) for value in rec_values]
                    f1_values = file.readline().strip().split(": ")[1].strip()[1:-1].split(", ")
                    f1_map[current_model] = [float(value) for value in f1_values]
                    tps_values = file.readline().strip().split(": ")[1].strip()[1:-1].split("], [")[1:-1]
                    tps_map[current_model] = [[int(val) for val in step.split(", ")] for step in tps_values]
                    tns_values = file.readline().strip().split(": ")[1].strip()[1:-1].split("], [")[1:-1]
                    tns_map[current_model] = [[int(val) for val in step.split(", ")] for step in tns_values]
                    fps_values = file.readline().strip().split(": ")[1].strip()[1:-1].split("], [")[1:-1]
                    fps_map[current_model] = [[int(val) for val in step.split(", ")] for step in fps_values]
                    fns_values = file.readline().strip().split(": ")[1].strip()[1:-1].split("], [")[1:-1]
                    fns_map[current_model] = [[int(val) for val in step.split(", ")] for step in fns_values]

    return avg_acc_map, avg_pre_map, avg_rec_map, avg_f1_map, acc_map, pre_map, rec_map, f1_map, tps_map, tns_map, fps_map, fns_map

# Example usage:
average_metrics_file = "./output/tl/average_metrics.txt"
avg_acc_map, avg_pre_map, avg_rec_map, avg_f1_map, acc_map, pre_map, rec_map, f1_map, tps_map, tns_map, fps_map, fns_map = read_metrics(average_metrics_file)


In [33]:
# for model in 
# accuracy_list, precision_list, recall_list, f1_list, tps, tns, fps, fns = train_net("googlenet")
model_names = [
    "resnet50",
    "resnet18", "alexnet", "vgg16",
    "densenet", "inception", "googlenet", 
    "resnext50_32x4d", "wide_resnet50_2"
]
acc_map = {}
pre_map = {}
rec_map = {}
f1_map = {}
tps_map = {}
tns_map = {}
fps_map = {}
fns_map = {}

for model_name in model_names:
    print(f"for {model_name}...")
    for timestamp in [1, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 289]:
        if "inception" in model_name:
            dataloaders = get_dataloaders(timestamp, inception=True)
        else:
            dataloaders = get_dataloaders(timestamp)
    
        model = load_model_for_timestamp(model_name, timestamp)
        display_heatmaps_for_timestamp(timestamp, model, model_name, dataloaders)

for resnet50...
timestamp:  1
Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)


100%|█████████████████████████████████████████████████████████████████| 64/64 [00:02<00:00, 21.91it/s]


Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)


100%|█████████████████████████████████████████████████████████████████| 64/64 [00:02<00:00, 21.89it/s]


Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)


...

100%|█████████████████████████████████████████████████████████████████| 64/64 [00:03<00:00, 16.87it/s]


Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)


100%|█████████████████████████████████████████████████████████████████| 64/64 [00:03<00:00, 19.24it/s]


for resnet18...
timestamp:  1


AttributeError: 'BasicBlock' object has no attribute 'conv3'